## Introduction
Diabetes remains a pervasive and important health concern within contemporary healthcare and medical research. Individuals with diabetes encounter challenges either in insulin production or the efficient utilization of insulin for glucose processing, resulting in potential complications such as cardiovascular issues and nerve damage. The multifaceted nature of this condition emphasizes the significance of research efforts aimed at explaining predictive factors for diabetes onset. Notably, existing medical literature has established correlations between diabetes and various risk factors, including obesity, age, and other demographic variables.

This project aims to address a pivotal question in the realm of diabetes research: Can the onset of diabetes be predicted based on an analysis of a patient's medical history and demographic data? To explore this question, we turn our attention to the diabetes prediction dataset meticulously curated by Mohammed Mustafa. Drawing from a global pool of medical and demographic data, this dataset contains a diverse array of features, including but not limited to age, body mass index (BMI), heart disease status, HbA1c level, and blood glucose level.

In the pursuit of answering our research question, we draw inspiration from relevant studies, such as the work by Brown et al. (2015), which explores predictive modeling in the context of diabetes, and the research by Diabetes Care (2005), delving into the prediction of diabetes development in older populations. These references serve as foundational pillars, guiding our approach and establishing connections between our project and established research in the field.


## Methods & Results

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import altair as alt
from sklearn.model_selection import cross_val_score

# Disables maximum rows allowed for altair plots
# alt.data_transformers.disable_max_rows()
# Uncomment below to re-enable max rows
# alt.data_transformers.enable('default', max_rows=5000)

In [18]:
url = "https://drive.google.com/file/d/1dTmTAiRGM5skZzMb9NwpOkcQrY0Dpq6t/view?usp=sharing"
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
diabetes = pd.read_csv(url) #read data
display(diabetes)
display(diabetes.info())
diabetes["diabetes"].value_counts(normalize = True) #show classification variable distribution

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


None

0    0.915
1    0.085
Name: diabetes, dtype: float64

Next we need to resample the data to create an even distribution of positive and negative labels.

In [19]:
np.random.seed(1) # set seed

diabetes_negative = diabetes[diabetes["diabetes"] == 0] #create even amounts of positive and negative labels
diabetes_positive = diabetes[diabetes["diabetes"] == 1]
diabetes_negative_downscaled = resample(
    diabetes_negative, n_samples = diabetes_positive.shape[0]
)
diabetes_negative_downscaled.shape[0]
diabetes_downsampled = pd.concat((diabetes_positive, diabetes_negative_downscaled))
display(diabetes_downsampled["diabetes"].value_counts(normalize = True))
diabetes_downsampled.info()

1    0.5
0    0.5
Name: diabetes, dtype: float64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17000 entries, 6 to 50426
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               17000 non-null  object 
 1   age                  17000 non-null  float64
 2   hypertension         17000 non-null  int64  
 3   heart_disease        17000 non-null  int64  
 4   smoking_history      17000 non-null  object 
 5   bmi                  17000 non-null  float64
 6   HbA1c_level          17000 non-null  float64
 7   blood_glucose_level  17000 non-null  int64  
 8   diabetes             17000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 1.3+ MB


Now that the data is resampled we can create the train/test split

In [20]:
diabetes_train, diabetes_test = train_test_split(
    diabetes_downsampled, train_size = .75, stratify = (diabetes_downsampled["diabetes"]), random_state=42 # split data
)
display(diabetes_train.info())
diabetes_train["diabetes"].value_counts(normalize = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12750 entries, 7874 to 68193
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               12750 non-null  object 
 1   age                  12750 non-null  float64
 2   hypertension         12750 non-null  int64  
 3   heart_disease        12750 non-null  int64  
 4   smoking_history      12750 non-null  object 
 5   bmi                  12750 non-null  float64
 6   HbA1c_level          12750 non-null  float64
 7   blood_glucose_level  12750 non-null  int64  
 8   diabetes             12750 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 996.1+ KB


None

0    0.5
1    0.5
Name: diabetes, dtype: float64

Now we filter the data to the numeric columns to aggregate and observe trends, and then do the same with categorical values using ``value_counts``

In [21]:
diabetes_stats_downsample = diabetes_downsampled.drop(["gender", "hypertension", "smoking_history", "diabetes"], axis=1) # find mean values
display(diabetes_stats_downsample.agg(["mean","std"]))
diabetes_stats = diabetes_train.drop(["gender", "hypertension", "smoking_history", "diabetes"], axis=1) # find mean values
display(diabetes_stats.agg(["mean","std"])) #show average + variability demographics for survey
#display(diabetes["gender"].value_counts(normalize = True))
#display(diabetes["hypertension"].value_counts(normalize = True))
#display(diabetes["smoking_history"].value_counts(normalize = True))

,age,heart_disease,bmi,HbA1c_level,blood_glucose_level
mean,50.535948,0.089294,29.432829,6.164053,163.706529
std,21.495405,0.285176,7.459182,1.283300,56.797977


,age,heart_disease,bmi,HbA1c_level,blood_glucose_level
mean,50.556235,0.088392,29.419256,6.167953,163.655137
std,21.530418,0.283876,7.493883,1.286576,56.873874


Next we'll make the preprocessor to use in K Means Classification

In [22]:
feature_names = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

diabetes_preprocessor = make_column_transformer(
    (StandardScaler(), feature_names),
)
diabetes_preprocessor

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['age', 'bmi', 'HbA1c_level',
                                  'blood_glucose_level'])])

In [23]:
diabetes_preprocessor.fit(diabetes)
diabetes_scaled = diabetes_preprocessor.transform(diabetes)
diabetes_scaled_df = pd.DataFrame(diabetes_scaled, columns=feature_names)
diabetes_scaled_df

,age,bmi,HbA1c_level,blood_glucose_level
0,1.692704,-0.321056,1.001706,0.047704
1,0.538006,-0.000116,1.001706,-1.426210
2,-0.616691,-0.000116,0.161108,0.489878
3,-0.261399,-0.583232,-0.492690,0.416183
4,1.515058,-1.081970,-0.679490,0.416183
...,...,...,...,...
99995,1.692704,-0.000116,0.628107,-1.180558
99996,-1.771388,-1.499343,0.908306,-0.934905
99997,1.070944,0.076729,0.161108,0.416183
99998,-0.794336,1.220361,-1.426688,-0.934905


Using our preprocessor, we can train a new model.

In [24]:
diabetes_pipe_knn = make_pipeline(diabetes_preprocessor, KNeighborsClassifier())

X_train = diabetes_train[["age", "bmi", "HbA1c_level", "blood_glucose_level"]]
y_train = diabetes_train["diabetes"]

cv_scores = cross_val_score(diabetes_pipe_knn, X_train, y_train, cv=5, scoring='accuracy')

cv_scores_std = np.std(cv_scores)
print("average cv score:", cv_scores.mean(), "±", cv_scores_std)

average cv score: 0.8830588235294119 ± 0.005887056942680853


Here, we see a score of __ . This is fine, however let's see if we can improve our score by tuning the `n_neighbors` hyperparameter in `KNeighborsClassifier()`.

### Hyperparameter Optimization
Now, we'll make a GridSearch CV object and a range of potential K values to find the best K value

In [25]:
diabetes_grid = {
    "kneighborsclassifier__n_neighbors"  : range(
        1,60, 2),
}
diabetes_pipe = make_pipeline(diabetes_preprocessor, KNeighborsClassifier())
diabetes_grid = GridSearchCV(
    estimator = diabetes_pipe,
    param_grid = diabetes_grid,
    cv = 5
)
accuracies_grid = pd.DataFrame(
    diabetes_grid.fit(
        diabetes_train[["age", "bmi", "HbA1c_level", "blood_glucose_level"]],
        diabetes_train["diabetes"],
    ).cv_results_
)
accuracies_grid = (
    accuracies_grid[[
        "param_kneighborsclassifier__n_neighbors",
        "mean_test_score",
        "std_test_score"
    ]]
    .assign(sem_test_score=accuracies_grid["std_test_score"] / 10**(1/2))
    .rename(columns={"param_kneighborsclassifier__n_neighbors": "n_neighbors"})
    .drop(columns=["std_test_score"])
)
accuracies_grid

,n_neighbors,mean_test_score,sem_test_score
0,1,0.873020,0.001770
1,3,0.877882,0.002248
2,5,0.883059,0.001862
3,7,0.887843,0.001569
4,9,0.888941,0.001294
5,11,0.892863,0.001753
6,13,0.894039,0.001591
7,15,0.893020,0.001550
8,17,0.892627,0.001156
9,19,0.893725,0.000976


In [26]:
accuracy = alt.Chart(accuracies_grid).mark_line(point=True).encode(
    x = alt.X("n_neighbors"),
    y = alt.Y("mean_test_score", title='Mean Test Score', scale=alt.Scale(zero=False)),
).properties(
    title='Ideal n_neighbors value based off Mean Test Score',
    width=600
)
accuracy

alt.Chart(...)

Based off `accuracies_grid` and our accuracy plot, we can see that `KNeighborsClassifier()` will perform best when `n_neighbors = 31` based off mean test score. Although `n_neighbors = 33` has a high `sem_test_score` such that there may be more variability in the mean test score, it will likely perform better than all other `n_neighbors` values up to `n_neighbors = 60`. There is a possibility of a better `n_neighbors` value past 60, however we will be unable to determine such value due to limited computational power.

In [27]:
diabetes_pipe_knn_31 = make_pipeline(diabetes_preprocessor, KNeighborsClassifier(n_neighbors=31))

X_train = diabetes_train[["age", "bmi", "HbA1c_level", "blood_glucose_level"]]
y_train = diabetes_train["diabetes"]

cv_scores = cross_val_score(diabetes_pipe_knn_31, X_train, y_train, cv=5, scoring='accuracy')

cv_scores_std = np.std(cv_scores)
print("average cv score:", cv_scores.mean(), "±", cv_scores_std)

average cv score: 0.8965490196078431 ± 0.0016711588825617852


After performing a cross-validation where cv=5, our model has an accuracy of 0.89655 with a standard deviation of 0.00167. A high accuracy may indicate that our model is able to correctly make predictions, and a low standard deviation indicates that our model is likely not underfitting nor overfitting. 

### Feature Selection
So far, we have found a feature set that allows us to train a relatively decent model. However, we would like to see whether there is another pool of features which may produce a more accurate model. To do this, we will separate our data into numerical features, binary features, and categorical features. We will also drop 'gender', as it may introduce bias into our final model. We could choose to represent 'gender' as a binary feature or an ordinal feature, but we believe it is best to not to include it at all.

We will pass all numerical features through `StandardScaler()` like previously, but this time we will pass categorical features (`smoking_history`) through `OrdinalEncoder()`, as we will choose to interpret smoking history as different levels of smoking intensity (never < former < current). However, we would also like to note that that 'no info' may affect our results, and using `OneHotEncoder()` is another option to consider in the future.

As for our binary features, we will use 'passthrough' as they do not need any additional processing to be used in `KNeighborsClassifier()`.

Lastly, we need to consider that the previous `n_neighbors=31` hyperparamater that we found is specific to the feature set containing only numerical features. Thus, we will use the default value of `n_neighbors=5` in our `KNeighborsClassifier()`.

In [28]:
X_train = diabetes_train.drop(columns=['diabetes', 'gender'])
y_train = diabetes_train['diabetes']
X_test = diabetes_train.drop(columns=['diabetes', 'gender'])
y_test = diabetes_train['diabetes']

numerical_features = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
binary_features = ['hypertension', 'heart_disease'] ## No need to perform any preprocessing
categorical_features = ['smoking_history', ]

preprocessor_all = make_column_transformer(
    (StandardScaler(), numerical_features),
    (OrdinalEncoder(), categorical_features),
    ('passthrough', binary_features)
)

X_train_transformed = preprocessor_all.fit_transform(X_train)
X_test_transformed = preprocessor_all.fit_transform(X_test)
y_train_transformed = y_train
y_test_transformed = y_test

pipe = make_pipeline(preprocessor_all, KNeighborsClassifier())

cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy')

cv_scores_std = np.std(cv_scores)
print("average cv score:", cv_scores.mean(), "±", cv_scores_std)

average cv score: 0.8898823529411766 ± 0.005531497312158574


Our score has decreased from our previous model. Let's see if we can select an ideal combinations of features by manually sifting through all combinations and finding the set with the best score.

In [29]:
from sklearn.model_selection import cross_val_score
from itertools import combinations

## Used ChatGPT to help iterate over all possible combinations of features as to select the top ten feature sets with the greatest score

def score_feature_combinations(X, y, model, preprocessors):
    scores = []

    for r in range(1, len(X.columns) + 1):
        for feature_combination in combinations(X.columns, r):
            preprocessor = make_column_transformer(
                (StandardScaler(), [feature for feature in feature_combination if feature in numerical_features]),
                (OrdinalEncoder(), [feature for feature in feature_combination if feature in categorical_features]),
                ('passthrough', [feature for feature in feature_combination if feature in binary_features])
            )

            pipe = make_pipeline(preprocessor, model)

            avg_score = cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()
            scores.append((feature_combination, avg_score))
    scores.sort(key=lambda x: x[1], reverse=True)

    return scores

knn_model = KNeighborsClassifier()
feature_combination_scores = score_feature_combinations(X_train, y_train, knn_model, preprocessor_all)

for feature_combination, score in feature_combination_scores[:10]:
    print(f"Feature Combination: {feature_combination}, Score: {score}")

Feature Combination: ('age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8909803921568628
Feature Combination: ('age', 'hypertension', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8904313725490196
Feature Combination: ('age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8898823529411766
Feature Combination: ('age', 'hypertension', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8889411764705881
Feature Combination: ('age', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8871372549019607
Feature Combination: ('age', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8864313725490197
Feature Combination: ('age', 'hypertension', 'smoking_history', 'HbA1c_level', 'blood_glucose_level'), Score: 0.8853333333333333
Feature Combination: ('age', 'hypertension', 'heart_disease', 'smoking_his

The feature combination of `'age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level', 'blood_glucose_level'` produces a model with a score of 0.89098, slightly outperforming our model which considers all features (0.88988) and the model which only considers only numerical features with hyperparameter optimization (0.88306).

There is other information we can infer from our feature combinations. `age, blood_glucose_level` and `HbA1c_level` seem to be the most important features to consider when predicting if a patient has diabetes as these three features all appear in top ten feature combinations. However, the addition of `hypertension`, `bmi` and/or `smoking_history` are necessary to help improve the accuracy of our models. 

In [30]:
X_train_opt = diabetes_train.drop(columns=['diabetes', 'gender', 'smoking_history'])
y_train_opt = diabetes_train['diabetes']
X_test_opt = diabetes_train.drop(columns=['diabetes', 'gender', 'smoking_history'])
y_test_opt = diabetes_train['diabetes']

numerical_features_opt = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
binary_features_opt = ['hypertension', 'heart_disease'] ## No need to perform any preprocessing

preprocessor_opt = make_column_transformer(
    (StandardScaler(), numerical_features_opt),
    ('passthrough', binary_features_opt)
)

X_train_transformed_opt = preprocessor_opt.fit_transform(X_train_opt)
X_test_transformed_opt = preprocessor_opt.fit_transform(X_test_opt)
y_train_transformed_opt = y_train_opt
y_test_transformed_opt = y_test_opt

pipe_opt = make_pipeline(preprocessor_opt, KNeighborsClassifier())

cv_scores_opt = cross_val_score(pipe_opt, X_train_opt, y_train_opt, cv=5, scoring='accuracy')

cv_scores_opt_std = np.std(cv_scores)
print("average cv score:", cv_scores_opt.mean(), "±", cv_scores_opt_std)

average cv score: 0.8909803921568628 ± 0.005531497312158574


To further improve our model accuracy, we could perform hyperparamater optimization on `n_neighbors`. However due to time constraints, we will continue working with our model using the default value of `n_neighbors=5`.

TODO:  compare test score with numerical features only model, use model on new observation(s), write a conclusion

In [31]:
#TODO: test scores for numerical features only, knn=31, and for optimized feature set
diabetes_pipe_knn_31.fit(X_train, y_train)

X_test = diabetes_test[["age", "bmi", "HbA1c_level", "blood_glucose_level"]]
y_test = diabetes_test["diabetes"]

test_score = diabetes_pipe_knn_31.score(X_test, y_test)
print("Test set accuracy:", test_score)

Test set accuracy: 0.8967058823529411


In [32]:
# TODO: new observation(s) for both numerical features only, and for optimized feature set, discuss findings and results

# new_observation = pd.DataFrame({
#     "age": [26.0],  
#     "bmi": [25.5],
#     "HbA1c_level": [1.8],
#     "blood_glucose_level": [170.0]
# })

# knn_fit = make_pipeline(diabetes_preprocessor, KNeighborsClassifier(n_neighbors=33)).fit(
#     diabetes_downsampled[feature_names],
#     diabetes_downsampled["diabetes"]
# )

# new_prediction = knn_fit.predict(new_observation)
# new_prediction

## Discussion

### Summary of Findings:

In our investigation to predict the onset of diabetes using a classification approach, we explored multiple feature combinations and their corresponding scores. Notably, the feature combination (`age`, `hypertension`, `heart_disease`, `bmi`, `HbA1c_level`, `blood_glucose_level`) emerged as the top performer with a score of 0.89098. This combination includes crucial variables such as age, hypertension, heart disease, BMI, HbA1c level, and blood glucose level.

### Expectations vs. Findings:

Our findings align with expectations to a considerable extent. The inclusion of age, hypertension, presence of heart disease, and indicators of metabolic health (BMI, HbA1c, blood glucose) is consistent with established medical knowledge linking these factors to diabetes risk. However, the presence of age, blood glucose level, and HbA1c level are most crucial since these variables appear in all top ten feature combinations.

### Impact of Findings:

Finding the best combination of features is important for predicting diabetes earlier on. Achieving a high score with the selected features highlights their predictive relevance, potentially enabling healthcare professionals to proactively identify individuals at risk. This can facilitate early intervention and lifestyle modifications, contributing to improved patient outcomes and reduced healthcare costs.

### Future Questions and Considerations:

Our findings creates opportunities for more in-depth exploration and improvement of models predicting the onset of diabetes. Future studies could look closely at how specific demographic and medical factors interact, aiming to find subtle details that make predictions more accurate. It is also valuable to check if the effectiveness of certain features changes over time or with different health habits.

Moreover, checking our results on different datasets can make our findings more widely applicable. For instance, bringing in genetic data and exploring new technologies such as wearables and continuous glucose monitoring could add more details to improve our predictive models.

In summary, our analysis offers valuable insights into which features predict diabetes early, suggesting the potential for timely intervention. The unexpected absence of certain variables leaves room for more questions, emphasizing the changing nature of predictive modeling in healthcare. As we navigate these findings, continuous research efforts can refine models and contribute to the ever-evolving field of predictive analytics in healthcare.

## References
Herman, W. H., Robinson, N., & Aubert, R. E. (2005). Predicting the Development of Diabetes in Older Adults. Diabetes Care, 28(2), 404-408. https://doi.org/10.2337/diacare.28.2.404

Zhang, P., Zhang, X., Brown, J., Vistisen, D., Sicree, R., Shaw, J., & Nichols, G. (2016). Big Data Approaches for Predicting the Onset of Type 2 Diabetes Mellitus in a Chinese Hospital Database. Diabetes Care, 39(7), 1095-1101. https://doi.org/10.2337/dc15-2042